In [10]:
from datasets import Dataset 
from ragas.metrics import faithfulness
from ragas import evaluate
import os
from datasets import Dataset
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.chat_models import ChatZhipuAI
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
import os
os.environ["ZHIPUAI_API_KEY"] = "661a7aa0aeb8ca129eb4647461123230.bl9w581QKpnMfBvs"
chat = ChatZhipuAI(
    model="glm-4-0520",
    temperature=0.5,
)


In [17]:
from langchain_community.chat_models import ChatTongyi

tongyi_chat = ChatTongyi(
    model="qwen-max-0428",
    # top_p="...",
    api_key="sk-a3905c2ac95d450d954b4a397244a7d9",
    # other params...
)

In [18]:
messages = [
    ("system", "你是一名专业的翻译家，可以将用户的中文翻译为英文。"),
    ("human", "我喜欢编程。"),
]
tongyi_chat.invoke(messages)

AIMessage(content='I enjoy programming.', response_metadata={'model_name': 'qwen-max-0428', 'finish_reason': 'stop', 'request_id': 'd60f23ba-5360-9039-a312-6e41c7399e12', 'token_usage': {'input_tokens': 30, 'output_tokens': 4, 'total_tokens': 34}}, id='run-8db69089-4e92-45af-90b9-d48480f297f8-0')

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

# ✅ Pydantic class
class multiply(BaseModel):
    """Return product of 'x' and 'y'."""
    x: float = Field(..., description="First factor")
    y: float = Field(..., description="Second factor")
    
# ✅ LangChain tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y
    
# ✅ Function

def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y-x
    
# ✅ OpenAI-format dict
# Could also pass in a JSON schema with "title" and "description" 
add = {
  "name": "add",
  "description": "Add 'x' and 'y'.",
  "parameters": {
    "type": "object",
    "properties": {
      "x": {"type": "number", "description": "First number to add"},
      "y": {"type": "number", "description": "Second number to add"}
    },
    "required": ["x", "y"]
  }
}

In [23]:
llm_with_tools = tongyi_chat.bind_tools([exponentiate, add, subtract])

In [ ]:
llm_with_tools.kwargs

In [27]:
query = 'what is 3 ** 12?'
tr = llm_with_tools.invoke(query).tool_calls

In [28]:
for tc in tr:
    print(tc)

{'name': 'exponentiate', 'args': {'x': 3, 'y': 12}, 'id': ''}


In [30]:
selected_tool = {"add":add, "multiply":multiply, "exponentiate": exponentiate}[tc['name'].lower()]

In [33]:
selected_tool.invoke(tc["args"])

531441.0